In [1]:
import os
import pandas as pd

In [3]:
# Cell 1: Check installations and data
import os
import pandas as pd

# Check if data files exist
data_path = "/Users/joehaldenby/Portfolio_Projects/Brazil_commerce_analysis/data"
print("Data files:", os.listdir(data_path))

# Try importing BigQuery
try:
    from google.cloud import bigquery
    print("✅ BigQuery library installed")
except ImportError:
    print("❌ Need to install: pip install google-cloud-bigquery")

# Check for Google Cloud authentication
try:
    client = bigquery.Client()
    print("✅ Google Cloud authenticated")
except Exception as e:
    print("❌ Authentication needed:", str(e))

Data files: ['olist_sellers_dataset.csv', 'product_category_name_translation.csv', 'olist_orders_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_order_payments_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_products_dataset.csv']
✅ BigQuery library installed
❌ Authentication needed: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.


In [19]:
import os
from google.cloud import bigquery

# Set the path to your service account key
credential_path = "/Users/joehaldenby/Portfolio_Projects/Brazil_commerce_analysis/credentials/brazil-commerce-analysis-226acdd2c335.json"

# Set environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

# Test connection
project_id = "brazil-commerce-analysis"

try:
    client = bigquery.Client(project=project_id)
    print("✅ Successfully authenticated with service account!")
    
    # Test by listing datasets
    datasets = list(client.list_datasets())
    print(f"Found {len(datasets)} existing datasets")
    
except Exception as e:
    print("❌ Authentication failed:", str(e))

✅ Successfully authenticated with service account!
Found 0 existing datasets


In [21]:
# Create a dataset for your ecommerce data
dataset_id = "Brazil_commerce"
dataset_ref = client.dataset(dataset_id)

try:
    # Try to create the dataset
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # or "EU" if you prefer
    dataset = client.create_dataset(dataset)
    print(f"✅ Created dataset: {dataset_id}")
except Exception as e:
    if "already exists" in str(e):
        print(f"✅ Dataset {dataset_id} already exists")
    else:
        print(f"❌ Error creating dataset: {e}")

# List your CSV files
data_path = "/Users/joehaldenby/Portfolio_Projects/Brazil_commerce_analysis/data"
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
print(f"\nFound {len(csv_files)} CSV files:")
for file in csv_files:
    print(f"  - {file}")

✅ Created dataset: Brazil_commerce

Found 9 CSV files:
  - olist_sellers_dataset.csv
  - product_category_name_translation.csv
  - olist_orders_dataset.csv
  - olist_order_items_dataset.csv
  - olist_customers_dataset.csv
  - olist_geolocation_dataset.csv
  - olist_order_payments_dataset.csv
  - olist_order_reviews_dataset.csv
  - olist_products_dataset.csv


In [23]:
import pandas as pd
from google.cloud import bigquery

# Define table mappings (CSV filename -> BigQuery table name)
table_mappings = {
    'olist_orders_dataset.csv': 'orders',
    'olist_customers_dataset.csv': 'customers', 
    'olist_sellers_dataset.csv': 'sellers',
    'olist_products_dataset.csv': 'products',
    'olist_order_items_dataset.csv': 'order_items',
    'olist_order_payments_dataset.csv': 'order_payments',
    'olist_order_reviews_dataset.csv': 'order_reviews',
    'olist_geolocation_dataset.csv': 'geolocation',
    'product_category_name_translation.csv': 'category_translation'
}

# Upload each file
for csv_file, table_name in table_mappings.items():
    print(f"\n📤 Uploading {csv_file} -> {table_name}...")
    
    # Read CSV
    df = pd.read_csv(f"{data_path}/{csv_file}")
    
    # Upload to BigQuery
    table_id = f"{client.project}.Brazil_commerce.{table_name}"
    
    job = client.load_table_from_dataframe(df, table_id)
    job.result()  # Wait for job to complete
    
    print(f"✅ Uploaded {len(df):,} rows to {table_name}")

print("\n🎉 All tables uploaded successfully!")


📤 Uploading olist_orders_dataset.csv -> orders...
✅ Uploaded 99,441 rows to orders

📤 Uploading olist_customers_dataset.csv -> customers...
✅ Uploaded 99,441 rows to customers

📤 Uploading olist_sellers_dataset.csv -> sellers...
✅ Uploaded 3,095 rows to sellers

📤 Uploading olist_products_dataset.csv -> products...
✅ Uploaded 32,951 rows to products

📤 Uploading olist_order_items_dataset.csv -> order_items...
✅ Uploaded 112,650 rows to order_items

📤 Uploading olist_order_payments_dataset.csv -> order_payments...
✅ Uploaded 103,886 rows to order_payments

📤 Uploading olist_order_reviews_dataset.csv -> order_reviews...
✅ Uploaded 99,224 rows to order_reviews

📤 Uploading olist_geolocation_dataset.csv -> geolocation...
✅ Uploaded 1,000,163 rows to geolocation

📤 Uploading product_category_name_translation.csv -> category_translation...
✅ Uploaded 71 rows to category_translation

🎉 All tables uploaded successfully!


In [6]:
import os
import pandas as pd
from google.cloud import bigquery

# Use the correct credentials file path
credential_path = "/Users/joehaldenby/Portfolio_Projects/Brazil_commerce_analysis/credentials/brazil-commerce-analysis-226acdd2c335.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

# Create client
client = bigquery.Client()
print("✅ Client created successfully!")

# Check all tables in your dataset
tables = client.list_tables("Brazil_commerce")

print("\n📋 Tables in Brazil_commerce dataset:")
for table in tables:
    # Get table info
    table_ref = client.get_table(table)
    print(f"  ✅ {table.table_id}: {table_ref.num_rows:,} rows, {len(table_ref.schema)} columns")


✅ Client created successfully!

📋 Tables in Brazil_commerce dataset:
  ✅ category_translation: 71 rows, 2 columns
  ✅ customers: 99,441 rows, 5 columns
  ✅ geolocation: 1,000,163 rows, 5 columns
  ✅ order_items: 112,650 rows, 7 columns
  ✅ order_payments: 103,886 rows, 5 columns
  ✅ order_reviews: 99,224 rows, 7 columns
  ✅ orders: 99,441 rows, 8 columns
  ✅ products: 32,951 rows, 9 columns
  ✅ sellers: 3,095 rows, 4 columns
